In [1]:
# 作者：常晓松
# 作用：日志等级预测
# 时间： 2024/7/9 19:31
from imblearn.over_sampling import SMOTE
from scipy.sparse import diags
import scipy
import stellargraph as sg
from playsound import playsound
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import *
from keras import Model, Input
from keras import losses
from keras.callbacks import EarlyStopping
import torch
import os
import matplotlib.pyplot as plt
import datetime
import sklearn as sk
from sklearn.metrics import balanced_accuracy_score
from numpy import sort
from random import random
from stellargraph.layer.gcn import GraphConvolution, GatherIndices
from keras.layers import Lambda
from sklearn.preprocessing import LabelBinarizer
from scipy.spatial.distance import pdist
import pandas as pd
from tqdm import tqdm
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
import pymysql
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Flatten
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time
import pickle
from sklearn.model_selection import train_test_split  # 出留法
from keras.layers import Convolution1D, MaxPooling1D, LSTM
import random
import numpy
import math
from statistics import mean
from sklearn import svm, datasets
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
def get_ave_and_standard_error(data_list):

    SD_P = numpy.std(data_list, ddof=0)
    n = len(data_list)

    ave = mean(data_list)
    standard_error = SD_P / math.sqrt(n)
    return round(ave,3),round(standard_error,3)

def get_connection():
    global conn
    host = "127.0.0.1"
    user = "root"
    password = 'chang123'
    db = 'predict_log_final'
    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        charset='utf8',
        # autocommit=True,    # 如果插入数据，， 是否自动提交? 和conn.commit()功能一致。
    )
    return conn

def shuffle_data(node, semantic_vec, syntatic_vec,leaf_method_name_vec,lable, seed):
    c = list(zip(node, semantic_vec, syntatic_vec,leaf_method_name_vec,lable))  # 将a,b整体作为一个zip,每个元素一一对应后打乱
    random.seed(seed)
    random.shuffle(c)  # 打乱c
    node[:], semantic_vec[:], syntatic_vec[:],leaf_method_name_vec[:],lable[:] = zip(*c)  # 将打乱的c解开
    return node, semantic_vec, syntatic_vec,leaf_method_name_vec,lable



def get_method_name_emb(seq):
    cur = conn.cursor()
    sqli = "select method_name_emb from data_model_1 where seq=( select methodSeq from data_model_2 where seq=" + str(seq) + ")"
    result = cur.execute(sqli)
    oneRow = cur.fetchone()
    emb=oneRow[0]
    cur.close()
    return eval(emb)



def get_positive_data_from_db(vec_length):
    cur = conn.cursor()
    sqli = "select vectorStruct,logNum from data_model_2 where logNum<>0 and vectorStruct is not null;"
    result = cur.execute(sqli)
    methods_padding = np.empty(shape=[0, vec_length], dtype=int)
    labels = []
    for i in range(result):
        oneRow = cur.fetchone()
        methods_padding = string_vector_to_int(methods_padding, oneRow)
        labels.append(oneRow[1])
    return methods_padding, labels



def add_nagitive_data(methods_padding, lables):
    cur = conn.cursor()
    sqli = "select vectorStruct,logNum from data_model_2 where logNum=0 and vectorStruct is not null limit " + str(
        len(methods_padding)) + ";"
    result = cur.execute(sqli)
    for i in range(1, len(methods_padding)):
        oneRow = cur.fetchone()
        methods_padding = string_vector_to_int(methods_padding, oneRow)
        lables.append(oneRow[1])
    return methods_padding, lables


def string_vector_to_int(methods_padding, oneRow):
    methods_padding = np.append(methods_padding, [list(map(int, oneRow[0][1:len(oneRow[0]) - 1].split(",")))], axis=0)
    return methods_padding


def float_revert_int(value_list):
    revert_value_list = []
    for i in value_list:
        revert_value_list.append(int(np.round(i)))
    value_list = revert_value_list
    return revert_value_list


def draw_confusion_matrix(y_true, y_pred, dic_lables):

    plt.cla()

    labels = []
    for key in dic_lables:
        labels.append(key)
    print_message(type(labels))
    # 小数转整数
    y_pred_int = float_revert_int(y_pred)
    sns.set()
    f, ax = plt.subplots()
    C2 = confusion_matrix(y_true,y_pred,  labels=labels)
    print_message(C2)  # 打印出来看看
    sns.heatmap(C2, annot=True, fmt='.20g', ax=ax, cmap="YlGnBu")  # 画热力图
    ax.set_title('confusion matrix')  # 标题
    ax.set_xlabel('true')  # x轴
    ax.set_ylabel('predict')  # y轴
    save_pic(plt, '混淆矩阵')
    # plt.show()
    return plt


def save_pic(plt, file_name):
    # 创建目录
    dirs = 'C:\\Users\\chang\\Desktop\\日志工作空间\\实验图片\\'
    t = time.strftime('%Y-%m-%d-%H', time.localtime(int(time.time())))
    dirs = dirs + t
    file = dirs + '\\' + file_name + '.png'
    if not os.path.exists(dirs):
        os.makedirs(dirs)
    if os.path.exists(file):
        os.remove(file)
    # 保存图片
    plt.savefig(file)  # 保存图片


def plot_value(y_true, y_pred):
    balance_accuracy = balanced_accuracy_score(y_true, y_pred)
    precision = sk.metrics.precision_score(y_true, y_pred)
    recall = sk.metrics.recall_score(y_true, y_pred)
    f1_value = sk.metrics.f1_score(y_true, y_pred)
    return balance_accuracy, precision, recall, f1_value


def autolabel(rects, ax):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


def plot_graphs(history_loc, type_loc):
    plt.cla()

    plt.plot(history_loc.history[type_loc])
    plt.plot(history_loc.history['val_' + type_loc])
    plt.xlabel('Epochs')
    plt.ylabel(type_loc)
    plt.legend([type_loc, 'val_' + type_loc])
    save_pic(plt, type_loc)
    # plt.show()
    return plt


def str_revert_to_list(methods_padding, max_len):
    methods_padding_list = np.empty(shape=[0, max_len], dtype=int)

    for methods_padding_one in methods_padding:
        list1 = methods_padding_one.split(',')
        list1 = list(map(int, list1))
        methods_padding_list = np.append(methods_padding_list, [np.array(list1)], axis=0)  # 添加整行元素，axis=1添加整列元素
    return methods_padding_list


def get_object(save_absolute_path):
    summer_load = None
    with open(save_absolute_path, 'rb') as f:
        summer_load = pickle.load(f)  # read file and build object
    return summer_load


def save_object(summer_save, save_absolute_path):
    summer_save = pickle.dumps(summer_save)
    with open(save_absolute_path, 'wb') as f:  # open file with write-mode
        f.write(summer_save)  # serialize and save object


def get_parent(vec_chair):
    cur = conn.cursor()
    sqli = "select parentId from data_model_2 where seq=" + str(vec_chair[len(vec_chair) - 1]) + ";"
    result = cur.execute(sqli)
    oneRow = cur.fetchone()
    if oneRow[0] == 0:
        return vec_chair
    else:
        vec_chair.append(oneRow[0])
        return get_parent(vec_chair)


def get_chair_vec(vec_chair, i,maxlen):
    cur = conn.cursor()
    one_leaf_list = []
    leaf = True
    message_type = i
    for one_leaf in vec_chair:
        sqli = "select a.vectorSemantic,REPLACE(REPLACE(vectorStruct,'[',''),']',''),logNum ,REPLACE(REPLACE(syntacticMessage,'[',''),']','') " \
               "from data_model_2 a  where seq=" + str(one_leaf) + ";"
        result = cur.execute(sqli)
        oneRow = cur.fetchone()
        vectorSemantic = oneRow[0]
        vectorStruct = oneRow[1]
        logNum = oneRow[2]
        syntacticMessage = oneRow[3]
        if vectorSemantic != "[]":
            if message_type == 1 or message_type == 3:
                if vectorSemantic is not None :
                    if len(vectorSemantic)==0:
                        vectorSemantic='0'
                    list_semantic = vectorSemantic.split(',')
                    list_semantic = list(map(float, list_semantic))
                    list_semantic_tmp = pad_sequences([list_semantic], maxlen=maxlen,
                                                      padding="post", truncating="post", dtype='float32')
                    list_semantic = list_semantic_tmp[0]
                    one_leaf_list.extend(list_semantic)
                    # one_leaf_list.extend([0,0,0,0])
            if message_type == 2:
                if syntacticMessage is not None :
                    if len(syntacticMessage)==0:
                        syntacticMessage='0'
                    list_syntactic = syntacticMessage.split(',')
                    list_syntactic = list(map(float, list_syntactic))
                    list_syntactic_tmp = pad_sequences([list_syntactic], maxlen=maxlen,
                                                       padding="post", truncating="post", dtype='float32')
                    list_syntactic = list_syntactic_tmp[0]
                    one_leaf_list.extend(list_syntactic)
    cur.close()
    return one_leaf_list


def print_message(message):
    t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(time.time())))
    print('时间：' + t, end=" ", flush=True)
    print(message, flush=True)



def repetition_list(a):
    repetition = 6
    c = []
    for a_one in a:
        b = []
        for a_single in a_one:
            for i in range(repetition):
                b.append(a_single)
        c.append(b)
    return c


def overlap_channel(a, b):
    element_num = 0
    height = 0
    width = 0
    if len(a.shape) == 2:
        element_num = a.shape[0]
        height = 1
        width = a.shape[1]
    elif len(a.shape) == 3:
        element_num = a.shape[0]
        height = a.shape[1]
        width = a.shape[2]
    else:
        raise Exception(print_message('通道叠加 不支持这类维度:' + str(a.shape)))

    a = a.reshape(element_num, 1, height, width)
    b = b.reshape(element_num, 1, height, width)
    a = a.transpose(1, 0, 2, 3)
    b = b.transpose(1, 0, 2, 3)
    c = np.vstack((a, b))
    c = c.transpose(1, 2, 3, 0)
    return c


def check_list_exist(A, B):
    exist = False
    for row in A:
        if (row == B).all():
            exist = True
    return exist


def get_convert_data(A, B):
    C = []
    for A_one in A:
        for B_one in B:
            if (np.array(A_one) == np.array(B_one)).all():
                C.append(A_one)
    return C


def cosDist(A, B):
    return pdist(np.vstack([A, B]), 'cosine')  # np.sqrt(sum(np.power((A - B), 2)))


def get_normal_data(data_point, data_lable):
    neigh_num_a = 5
    diff_rate_b = 1
    can_stop = 0.99

    # print_message(len(data_point))

    abnormal_old = []
    abnormal_new_ = []
    for i in range(0, 100):
        for one_point, one_lable in zip(data_point, data_lable):
            neighbour_point = []
            neighbour_lable = []
            for one_point_neighbour, one_lable_neighbour in zip(data_point, data_lable):
                if check_list_exist(abnormal_new_, one_point_neighbour) or (one_point_neighbour == one_point).all():
                    continue
                else:
                    this_point_distance = cosDist(one_point_neighbour, one_point)
                    if len(neighbour_point) >= neigh_num_a \
                            and this_point_distance < neighbour_point[neigh_num_a - 1] \
                            or len(neighbour_point) < neigh_num_a:
                        if len(neighbour_point) < neigh_num_a:
                            neighbour_point.append(this_point_distance)
                            neighbour_lable.append(one_lable_neighbour)
                        else:
                            neighbour_point[neigh_num_a - 1] = (this_point_distance)
                            neighbour_lable[neigh_num_a - 1] = (one_lable_neighbour)
                        # 按照距离排序
                        zipped = zip(neighbour_point, neighbour_lable)
                        sort_zipped = sorted(zipped)
                        result = zip(*sort_zipped)
                        neighbour_point, neighbour_lable = [list(x) for x in result]

            same_num = 0
            diff_num = 0
            for one_point_neighbour, one_lable_neighbour in zip(neighbour_point[0:neigh_num_a],
                                                                neighbour_lable[0:neigh_num_a]):
                if one_lable_neighbour != one_lable:
                    diff_num += 1
                else:
                    same_num += 1
            diff_rate = diff_num / (same_num + diff_num)
            if diff_rate >= diff_rate_b:
                abnormal_new_.append([one_point])
        if max(len(abnormal_new_), len(abnormal_old)) != 0 and \
                len(get_convert_data(abnormal_new_, abnormal_old)) / max(len(abnormal_new_),
                                                                         len(abnormal_old)) >= can_stop:
            break
        else:
            abnormal_old = abnormal_new_[:]
        # print_message('单次noise数据量：' + str(len(abnormal_old)))

    data_point_clear = []
    data_lable_clear = []

    for one_point, one_lable in zip(data_point, data_lable):
        if (not check_list_exist(abnormal_old, one_point)):
            data_point_clear.append(one_point)
            data_lable_clear.append(one_lable)
    return data_point_clear, data_lable_clear, abnormal_old


##仅支持0 - 1 分类
def change_abnormal(data_point, data_lable):
    neigh_num_a = 10
    diff_rate_b = 1
    can_stop = 0.99

    # print_message(len(data_point))

    abnormal_old = []
    abnormal_new_ = []
    for i in range(0, 100):
        for one_point, one_lable in zip(data_point, data_lable):
            neighbour_point = []
            neighbour_lable = []
            for one_point_neighbour, one_lable_neighbour in zip(data_point, data_lable):
                if check_list_exist(abnormal_new_, one_point_neighbour) or (one_point_neighbour == one_point).all():
                    continue
                else:
                    this_point_distance = cosDist(one_point_neighbour, one_point)
                    if len(neighbour_point) >= neigh_num_a \
                            and this_point_distance < neighbour_point[neigh_num_a - 1] \
                            or len(neighbour_point) < neigh_num_a:
                        if len(neighbour_point) < neigh_num_a:
                            neighbour_point.append(this_point_distance)
                            neighbour_lable.append(one_lable_neighbour)
                        else:
                            neighbour_point[neigh_num_a - 1] = (this_point_distance)
                            neighbour_lable[neigh_num_a - 1] = (one_lable_neighbour)
                        # 按照距离排序
                        zipped = zip(neighbour_point, neighbour_lable)
                        sort_zipped = sorted(zipped)
                        result = zip(*sort_zipped)
                        neighbour_point, neighbour_lable = [list(x) for x in result]

            same_num = 0
            diff_num = 0
            for one_point_neighbour, one_lable_neighbour in zip(neighbour_point[0:neigh_num_a],
                                                                neighbour_lable[0:neigh_num_a]):
                if one_lable_neighbour != one_lable:
                    diff_num += 1
                else:
                    same_num += 1
            diff_rate = diff_num / (same_num + diff_num)
            if diff_rate >= diff_rate_b:
                abnormal_new_.append([one_point])
        if max(len(abnormal_new_), len(abnormal_old)) != 0 and \
                len(get_convert_data(abnormal_new_, abnormal_old)) / max(len(abnormal_new_),
                                                                         len(abnormal_old)) >= can_stop:
            break
        else:
            abnormal_old = abnormal_new_[:]
        # print_message('单次noise数据量：' + str(len(abnormal_old)))

    data_point_clear = []
    data_lable_clear = []

    for one_point, one_lable in zip(data_point, data_lable):
        if (not check_list_exist(abnormal_old, one_point)):
            data_point_clear.append(one_point)
            data_lable_clear.append(one_lable)
        else:
            ##修改noise标签
            data_point_clear.append(one_point)
            if one_lable == 0:
                data_lable_clear.append(1)
            else:
                data_lable_clear.append(0)
    return data_point_clear, data_lable_clear, abnormal_old


def clear_data(train_vec_tmp, train_labels, zero_index):
    group_num = 300
    train_vec_tmp_clear = []
    train_labels_clear = []
    abnormal_num = 0
    for b in range(int(len(train_vec_tmp) / group_num) + 1):
        begin = b * group_num + zero_index
        end = (b + 1) * group_num
        if end > len(train_vec_tmp):
            end = len(train_vec_tmp)
        train_vec_group_one, train_labels_group_one, abnormal_old = change_abnormal(train_vec_tmp[begin:end],
                                                                                    train_labels[begin:end])
        train_vec_tmp_clear.extend(train_vec_group_one)
        train_labels_clear.extend(train_labels_group_one)
        abnormal_num = abnormal_num + len(abnormal_old)
    print_message('noise数据量：' + str(abnormal_num))
    print_message('训练总数据量：' + str(len(train_vec_tmp)))
    train_vec_tmp = train_vec_tmp_clear
    train_labels = train_labels_clear
    return train_vec_tmp, train_labels


def play_title_message():
    playsound('D:/python/log_predict/file/结束提示.mp3')


def get_tree_model_together(struct_vec_length_one,semantic_vec_length_one,syntatic_vec_length_one
                            ,num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        keras.layers.InputLayer(struct_vec_length_one)
    ])
    model2 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(semantic_vec_length_one, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(syntatic_vec_length_one, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model_together = keras.layers.concatenate([model3.output, model2.output])
    x = Dropout(0.4)(model_together)
    x = Dense(24, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    final_output = Dense(24, activation='relu')(x)
    model_together = Model(inputs=[model3.input, model2.input], outputs=final_output)
    return model_together



def seperate_data(struct_leaf_vec, semantic_leaf_vec, syntatic_leaf_vec,
                  lable, leaf,seed):
    struct_train_vec, struct_test_vec, \
    semantic_train_vec, semantic_test_vec, \
    syntatic_train_vec, syntatic_test_vec, \
    struct_train_labels, struct_test_labels, \
    struct_train_leaf, struct_test_leaf = train_test_split(
        struct_leaf_vec, semantic_leaf_vec, syntatic_leaf_vec,
        lable, leaf,
        test_size=0.2,
        random_state=seed)
    struct_verify_vec, struct_test_vec, \
    semantic_verify_vec, semantic_test_vec, \
    syntatic_verify_vec, syntatic_test_vec, \
    struct_verify_labels, struct_test_labels, \
    struct_verify_leaf, struct_test_leaf = train_test_split(
        struct_test_vec, semantic_test_vec, syntatic_test_vec,
        struct_test_labels,
        struct_test_leaf,
        test_size=0.5,
        random_state=seed)
    return struct_train_vec, struct_test_vec, \
           semantic_train_vec, semantic_test_vec, \
           syntatic_train_vec, syntatic_test_vec, \
           struct_train_labels, struct_test_labels, \
           struct_verify_vec, semantic_verify_vec, \
           syntatic_verify_vec, struct_verify_labels, struct_verify_leaf

def get_lable(conn, seq):
    seq_db = 0
    leaf = ''
    lable = 0
    sqli = "select seq,leaf,CASE WHEN logNum > 0 THEN  1  ELSE 0 END from data_model_2 where seq=" + seq
    cur = conn.cursor()
    result = cur.execute(sqli)
    for i in range(result):
        oneRow = cur.fetchone()
        seq_db = oneRow[0]
        leaf = oneRow[1]
        lable = oneRow[2]
    cur.close()
    return seq_db, leaf, lable



def get_feature_from_file(file_name, node_all, node_vec):
    with open(file_name, 'r') as file:
        line = file.readline()
        while line:
            one_row = line.split(' ')
            if len(one_row) > 2:
                node_all.append(one_row[0])
                one_row[len(one_row) - 1] = one_row[len(one_row) - 1].replace('\n', '')
                one_row = list(map(float, one_row))
                node_vec.append(list(map(float, one_row[1:])))
            line = file.readline()
    return node_all, node_vec


def convert_metric_to_list(project_name, feature_type, percentage, metric_type, tool_type):
    one_row = []
    one_row.append(project_name)
    one_row.append(feature_type)
    one_row.append(percentage)
    one_row.append(metric_type)
    one_row.append(tool_type)
    return one_row




def get_call_graph(graph_leng, num_word, embedding_dim):
    model3 = tf.keras.models.Sequential([
        Convolution1D(input_shape=(graph_leng, 1), filters=32, kernel_size=5, strides=1, padding='same',
                      activation='relu', name='call-message'),
        MaxPooling1D(pool_size=2, strides=2, padding='same', ),
        Convolution1D(64, 5, strides=1, padding='same', activation='relu'),
        MaxPooling1D(2, 2, 'same'),
        Flatten(),
        Dense(69, activation='relu'),
        Dense(2, activation='softmax'),
    ])
    return model3

def get_method_name_model(method_name_x, method_name_y, num_word, embedding_dim):

    model1 = tf.keras.models.Sequential([
        keras.layers.Masking(mask_value=0, input_shape=(method_name_x, method_name_y)),
        LSTM(units=8,activation='sigmoid'),
        #         LSTM(units=32, activation='tanh'),
        #         Flatten(),
        Dense(53, activation='relu'),
        Dense(2, activation='softmax'),
    ])
    return model1

def get_tree_model_together(semantic_vec_length_chain,syntatic_vec_length_chain,num_word,embedding_dim):
    model2 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(semantic_vec_length_chain, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        (Convolution1D(input_shape=(syntatic_vec_length_chain, 1), filters=1,
                       kernel_size=1, strides=1, padding='same',
                       activation='relu', kernel_initializer=keras.initializers.Ones(),
                       bias_initializer='zeros')),
        (MaxPooling1D(pool_size=2, strides=2, padding='same', )),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
        #         LSTM(units=32, activation='tanh', return_sequences=True),
        #         Flatten(),
    ])
    model_together = keras.layers.concatenate([model3.output, model2.output])
    x = Dropout(0.2)(model_together)
    final_output = Dense(43, activation='relu')(x)
    model_together = Model(inputs=[model3.input, model2.input], outputs=final_output)
    return model_together


def get_method_name_call_graph_model(method_dem_x,method_dem_y,call_graph_len,num_word,embedding_dim):
    method_name_model = get_method_name_model(method_dem_x,method_dem_y,  num_word, embedding_dim)
    call_graph_model = get_call_graph(call_graph_len,  num_word, embedding_dim)
    m_j_merged_a = keras.layers.concatenate([method_name_model.output,call_graph_model.output],axis=-1)
    dense1_a = Dense(4,activation='tanh')(m_j_merged_a)
    final_output = Dropout(0.2)(dense1_a)
    model_together_final = Model(inputs=[ method_name_model.input,call_graph_model.input],outputs=final_output)
    return model_together_final


def get_featrue_and_label(project_name):
    # 1-语法 2-语义 3-结合(本树+调用关系)
    # 4-语法（新）5-纯调用关系 6-结合(本树)
    select_model = 6
    semantic_vec_length = 50
    tree_deep=1
    semantic_vec_length_chain =  semantic_vec_length* tree_deep
    syntatic_vec_length_chain = 30 * tree_deep
    # 获取每个叶子节点
    node = []
    lable = []
    semantic_leaf_one_node = []
    semantic_leaf_vec = []
    syntatic_leaf_vec = []
    theme_leaf_vec = []

    conn = get_connection()
    cur = conn.cursor()

    sqli = "select seq,(select method_name_emb from data_model_1 where seq=a.methodSeq)," \
           "CASE WHEN logNum > 0 THEN  1  ELSE 0 END " \
           "from  data_model_2 a where  methodSeq in " \
           "(select seq from data_model_1 where projectName='"+project_name+"') "
    result = cur.execute(sqli)
    #  and methodSeq<59900
    for i in range(result):
        oneRow = cur.fetchone()
        node.append(oneRow[0])
        theme_leaf_vec.append(eval(oneRow[1]))
        lable.append(oneRow[2])
    print_message("获取节点特征")
    for one_leaf in node:
        vec_chain = []
        for i in range(1, 3):
            if i == 1:
                maxlen = semantic_vec_length_chain
                vec_chain = []
                vec_chain.append(one_leaf)
#                 vec_chain = get_parent(vec_chain)
            elif i==2:
                maxlen = syntatic_vec_length_chain
                vec_chain = []
                vec_chain.append(one_leaf)
                #vec_chain = get_parent(vec_chain)

            # 根据list中的记录查询节点记录，组合向量
            one_leaf_vec = get_chair_vec(vec_chain, i,maxlen//tree_deep)
            # 划分为等长向量
            one_leaf_vec_padding_tmp = pad_sequences([one_leaf_vec], maxlen=maxlen,
                                                     padding="post", truncating="post", dtype='float32')
            one_leaf_vec_padding_tmp = one_leaf_vec_padding_tmp[0]

            if i == 1:
                semantic_leaf_vec.append(one_leaf_vec_padding_tmp)
            elif i==2:
                syntatic_leaf_vec.append(one_leaf_vec_padding_tmp)

    cur.close()
    conn.close()
    return node,semantic_leaf_vec,syntatic_leaf_vec,theme_leaf_vec,lable


def get_feature_from_file(file_name, node_all, node_vec):
    with open(file_name, 'r') as file:
        line = file.readline()
        while line:
            one_row = line.split(' ')
            if len(one_row) > 2:
                node_all.append(one_row[0])
                one_row[len(one_row) - 1] = one_row[len(one_row) - 1].replace('\n', '')
                one_row = list(map(float, one_row))
                node_vec.append(list(map(float, one_row[1:])))
            line = file.readline()
    return node_all, node_vec



def build_data(node,semantic_vec,edge,lable):
    features_matrix_tmp=[]
    for one_node,feature_value in zip(node,semantic_vec):
        features_matrix_tmp.append(feature_value)

    features_matrix=np.array(features_matrix_tmp)
    a = {}
    index=0
    for i in features_matrix:
        a[index]=[int(i) for i in i.tolist()]
        index+=1


    max_feature = np.max([v for v_list in a.values() for v in v_list])
    features_matrix = np.zeros(shape = (len(list(a.keys())), max_feature+1))

    i = 0
    for k, vs in tqdm(a.items()):
        for v in vs:
            features_matrix[i, v] = 1
        i+=1



    class data:
        x=[]
        y=[]
        edge_index=[]
        train_mask=[]
        val_mask=[]
        test_mask=[]

    data=data()
    node=node
    label=lable
    edge=edge
    feature=[]

    feature = features_matrix
    # for i in node:
    #     feature.append(seq_feature[i])

    train_rate=0.6
    val_rate=0.2
    test_rate=0.2
    number=0
    node_new={}
    for i in node:
        node_new[i]=number
        number=number+1
    data.x=torch.Tensor(feature)
    edge_new=[[],[]]
    for i in edge:
        edge_new[0].append(node_new[i[0]])
        edge_new[1].append(node_new[i[1]])

    data.edge_index=torch.LongTensor(edge_new)

    data.y=torch.LongTensor(label)
    train_num=int(len(node)*train_rate)
    train_mask=[ False for i in node]
    while train_num>0:
        index=0
        for i in node:
            if train_num>0 and random.random()>=0.5 and not train_mask[index]:
                train_mask[index]=True
                train_num=train_num-1
            index=index+1
    data.train_mask=train_mask
    test_num=int(len(node)*test_rate)

    test_mask=[ False for i in node]
    while test_num>0:
        index=0
        for i,j in zip(test_mask,train_mask):
            if test_num>0 and not i and not j:
                if random.random()>=0.5:
                    test_mask[index]=True
                    test_num=test_num-1
            index=index+1

    data.test_mask=test_mask
    val_num=int(len(node)*val_rate)

    val_mask=[ False for i in node]
    while val_num>0:
        index=0
        for i,j,z in zip(val_mask,train_mask,test_mask):
            if val_num>0 and not i and not j and not z:
                if random.random()>=0.5:
                    val_mask[index]=True
                    val_num=val_num-1
            index=index+1
    data.val_mask=val_mask

    return data

def get_node_indices(G, ids):
    # find the indices of the nodes
    node_ids = np.asarray(ids)
    flat_node_ids = node_ids.reshape(-1)

    flat_node_indices = G.node_ids_to_ilocs(flat_node_ids) # in-built function makes it really easy
    # back to the original shape
    node_indices = flat_node_indices.reshape(1, len(node_ids)) # add 1 extra dimension

    return node_indices

def splite_data(vec_x_sem,vec_x_syn,vec_x_graph,vec_theme,
                vec_y,mask):
    vec_sem=[]
    vec_syn=[]
    vec_graph=[]
    vec_method_theme=[]
    lable=[]
    for i,j,h,w,g,z  in zip(vec_x_sem,vec_x_syn,
                              vec_x_graph,vec_theme,
                              vec_y,
                              mask):
        if z:
            vec_sem.append(i)
            vec_syn.append(j)
            vec_graph.append(h)
            vec_method_theme.append(w)
            lable.append(g)
    return vec_sem,vec_syn,vec_graph,vec_method_theme,lable
def get_method_name_emb(seq):
    cur = conn.cursor()
    sqli = "select method_name_emb from data_model_1 where seq=( select methodSeq from data_model_2 where seq=" + str(seq) + ")"
    result = cur.execute(sqli)
    oneRow = cur.fetchone()
    emb=oneRow[0]
    cur.close()
    return eval(emb)


# 得到多分枝模型
def get_graph_model_4(graph_leng,syntactic_len,method_name_x, method_name_y,
                      num_word,embedding_dim):
#     model1 = tf.keras.models.Sequential([
#         Input(shape=(semantic_vec,1)),
#         (Flatten()),
#         (keras.layers.Embedding(num_word, embedding_dim)),
#         (keras.layers.GlobalMaxPool1D()),
#     ])
    model2 = tf.keras.models.Sequential([
        Input(shape=(graph_leng,1)),
        Flatten(),
    ])
    model3 = tf.keras.models.Sequential([
        Input(shape=(syntactic_len,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model4 = tf.keras.models.Sequential([
        keras.layers.Masking(mask_value=0, input_shape=(method_name_x, method_name_y)),
        LSTM(units=8, activation='tanh', return_sequences=True),
        keras.layers.GlobalMaxPool1D(),
        
    ])
    model_together = keras.layers.concatenate([ model2.output, model3.output,model4.output])
    model_together=Lambda(lambda x:keras.backend.expand_dims(x,axis=1))(model_together)
    x = LSTM(units=32, activation='tanh', return_sequences=True)(model_together)
    x=keras.layers.GlobalMaxPool1D()(x)
    final_output = Dropout(0.2)(x)
    final_output = Dense(2, activation='softmax')(final_output)
    model_together = Model(inputs=[model2.input, model3.input,model4.input], outputs=final_output)
    return model_together

def get_graph_model_3(graph_leng,syntactic_len,
                      num_word,embedding_dim):
    model2 = tf.keras.models.Sequential([
        Input(shape=(graph_leng,1)),
        Flatten(),
    ])
    model3 = tf.keras.models.Sequential([
        Input(shape=(syntactic_len,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model_together = keras.layers.concatenate([ model2.output, model3.output])

    model_together=Lambda(lambda x:keras.backend.expand_dims(x,axis=1))(model_together)
    x = LSTM(units=32, activation='tanh', return_sequences=True)(model_together)
    x=keras.layers.GlobalMaxPool1D()(x)
    final_output = Dropout(0.2)(x)
    final_output = Dense(2, activation='softmax')(final_output)
    model_together = Model(inputs=[model2.input, model3.input], outputs=final_output)
    return model_together

def get_graph_model_2(semantic_vec,syntactic_len,
                      num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(semantic_vec,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model3 = tf.keras.models.Sequential([
        Input(shape=(syntactic_len,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        (keras.layers.GlobalMaxPool1D()),
    ])
    model_together = keras.layers.concatenate([model1.output, model3.output])
    model_together=Lambda(lambda x:keras.backend.expand_dims(x,axis=1))(model_together)

    x = LSTM(units=32, activation='tanh', return_sequences=True)(model_together)
    x=keras.layers.GlobalMaxPool1D()(x)
    final_output = Dropout(0.2)(x)
    final_output = Dense(2, activation='softmax')(final_output)
    model_together = Model(inputs=[model1.input,model3.input], outputs=final_output)
    return model_together
def get_graph_model_1(semantic_vec,
                      num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(semantic_vec,1)),
        (Flatten()),
        (keras.layers.Embedding(num_word, embedding_dim)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
        (Dropout(0.2)),
        Dense(2, activation='softmax'),
    ])
    return model1
def get_graph_model_gcn(graph_vec_len,
                        num_word,embedding_dim):
    model1 = tf.keras.models.Sequential([
        Input(shape=(graph_vec_len,1)),
        LSTM(units=32, activation='tanh', return_sequences=True),
        (keras.layers.GlobalMaxPool1D()),
        (Dropout(0.2)),
        Dense(2, activation='softmax'),

    ])
    return model1
def save_performance(content,ac,pr,re,f1):
    f = open("print-message.txt", mode='a')
    f.write(str(datetime.datetime.now())+'\t'+ content+' \t'+str(ac)+
            '\t'+str(pr)+ '\t'+str(re)+'\t'+ str(f1)+'\n')
    f.close()

def convert_2_hot_vec(list):
    key = []
    for i in list:
        tmp = [int(i) for i in i[::2]]
        for j in tmp:
            if j not in key:
                key.append(j)
        # print(i[1::2])
    # 得到one-hot表格
    key = sort(key)
    feature_hot = []
    for i in list:
        key_tmp = i[::2]
        num_tmp = i[1::2]
        key_tmp = [int(i) for i in key_tmp]
        num_tmp = [int(i) for i in num_tmp]
        
        feature_tmp = []
        for j in key:
            if j in key_tmp:
                feature_tmp.append(num_tmp[key_tmp.index(j)])
            else:
                feature_tmp.append(0)
        feature_hot.append(feature_tmp)
    return key,feature_hot
def plot_value(y_true, y_pred):
    import sklearn as sk
    from sklearn.metrics import balanced_accuracy_score
    balance_accuracy = balanced_accuracy_score(y_true, y_pred)
    precision = sk.metrics.precision_score(y_true, y_pred)
    recall = sk.metrics.recall_score(y_true, y_pred)
    f1_value = sk.metrics.f1_score(y_true, y_pred)
    return balance_accuracy, precision, recall, f1_value


In [2]:

num_epoch = 100  # 训练周期
feature_length = "10"
one_project = 'shardingsphere-elasticjob-master'
num_word=50000
embedding_dim=12

# 获取数据集

In [3]:

node, semantic_vec, \
syntatic_vec,\
leaf_method_name_vec,lable = get_featrue_and_label(one_project)

randomValue=int(random.random()*100)
node, semantic_vec, \
syntatic_vec,\
leaf_method_name_vec,lable=shuffle_data(node, semantic_vec,
                                        syntatic_vec,
                                        leaf_method_name_vec,lable,randomValue)


时间：2024-08-19 09:54:04 获取节点特征


In [4]:
number=0
node_new={}
for i in node:
    node_new[i]=number
    number=number+1

edge=[]#[[28,16],[16,85],[85,46],[28,16],[16,85],[85,46],[16,85],[85,46]]

conn = get_connection()

cur = conn.cursor()
sqli = "select parentId,seq " \
       "from data_model_2 " \
       "where methodSeq in " \
       "(select seq from data_model_1 where projectName='"+one_project+"') " \
       "and parentId<>0  " \
       "union " \
       "select callBlockSeq,calledBlockSeq " \
       "from call_graph_data " \
       "where callMethodSeq in " \
       "(select seq from data_model_1 where projectName='"+one_project+"')   "
result = cur.execute(sqli)
#  and methodSeq<59900  and callMethodSeq<59900 and calledMethodSeq<59900
for i in range(result):
    oneRow = cur.fetchone()
    if oneRow[0] in node_new.keys() and oneRow[1] in node_new.keys():
        edge_tmp=[]
        edge_tmp.append(node_new[oneRow[0]])
        edge_tmp.append(node_new[oneRow[1]])
        edge.append(edge_tmp)
cur.close()
conn.close()


In [5]:
save_object(node,one_project+'node.plk')
save_object(semantic_vec,one_project+'semantic_vec.plk')
save_object(syntatic_vec,one_project+'syntatic_vec.plk')
save_object(leaf_method_name_vec,one_project+'leaf_method_name_vec.plk')
save_object(lable,one_project+'lable.plk')
save_object(node_new,one_project+'node_new.plk')
save_object(edge,one_project+'edge.plk')

In [29]:
node=get_object('数据集-缓存//'+one_project+'node.plk')
semantic_vec=get_object('数据集-缓存//'+one_project+'semantic_vec.plk')
syntatic_vec=get_object('数据集-缓存//'+one_project+'syntatic_vec.plk')
leaf_method_name_vec=get_object('数据集-缓存//'+one_project+'leaf_method_name_vec.plk')
lable=get_object('数据集-缓存//'+one_project+'lable.plk')

# 组装特征

In [44]:
featrure_combined=[]
label_all=lable
for i,j,k in zip(semantic_vec,syntatic_vec,leaf_method_name_vec):
    one_feature=[]
    one_feature.extend(i)
    one_feature.extend(j)
    one_feature.extend(np.average(k, axis=0).tolist())
    featrure_combined.append(one_feature)
print('数据集构建成功')

数据集构建成功


In [ ]:
train_vec,test_vec,train_label,test_label = train_test_split(featrure_combined, 
                                                       label_all,test_size=0.2,
                                                       random_state=123)

# KNN

In [ ]:

ml = KNeighborsClassifier(n_neighbors=5, 
                          weights='uniform',
                          algorithm='auto',
                          leaf_size=80, p=9)    #实例化KNN模型     #放入训练数据进行训练
random_state = np.random.RandomState(0)
ml.fit(train_vec,train_label)
predict_number=ml.predict(test_vec)
verify_number=test_label
balance_accuracy, precision, recall, f1_value = plot_value(predict_number,verify_number)
save_performance(one_project+'-KNN',balance_accuracy, precision, recall, f1_value)

In [45]:
su = SMOTE(random_state=42)
featrure_combined, label_all = \
            su.fit_resample(featrure_combined, label_all)

In [46]:
train_vec,test_vec,train_label,test_label = train_test_split(featrure_combined, 
                                                       label_all,test_size=0.2,
                                                       random_state=123)

# SVM

In [48]:
kfold = model_selection.KFold(n_splits=10, shuffle=True,random_state=42)
# SVM
random_state = np.random.RandomState(0)
ml = svm.SVC(kernel='linear', probability=True,
             random_state=random_state,tol = 0.0001, max_iter = 100)
ml.fit(train_vec,train_label)
predict_number=ml.predict(test_vec)
verify_number=test_label
balance_accuracy, precision, recall, f1_value = plot_value(predict_number,verify_number)

save_performance(one_project+'-SVM',balance_accuracy, precision, recall, f1_value)

c:\users\chang\appdata\local\programs\python\python38\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


# 随机森林

In [49]:
random_state = np.random.RandomState(0)
ml = RandomForestClassifier(n_estimators=100, max_depth=1, random_state=456)
ml.fit(train_vec,train_label)
predict_number=ml.predict(test_vec)
verify_number=test_label
balance_accuracy, precision, recall, f1_value = plot_value(predict_number,verify_number)
save_performance(one_project+'-随机森林',balance_accuracy, precision, recall, f1_value)

# 朴素贝叶斯

In [50]:

ml = GaussianNB() #alpha拉普拉斯平滑系数，防止概率为0出现
random_state = np.random.RandomState(0)
ml.fit(train_vec,train_label)
predict_number=ml.predict(test_vec)
verify_number=test_label
balance_accuracy, precision, recall, f1_value = plot_value(predict_number,verify_number)
save_performance(one_project+'-朴素贝叶斯',balance_accuracy, precision, recall, f1_value)

# 决策树

In [51]:
ml= DecisionTreeClassifier(random_state=5,max_features = 3,max_leaf_nodes=3) # 初始化
random_state = np.random.RandomState(0)
ml.fit(train_vec,train_label)
predict_number=ml.predict(test_vec)
verify_number=test_label
balance_accuracy, precision, recall, f1_value = plot_value(predict_number,verify_number)
save_performance(one_project+'-决策树',balance_accuracy, precision, recall, f1_value)

# 逻辑回归

In [52]:
ml = LogisticRegression()
random_state = np.random.RandomState(0)
ml.fit(train_vec,train_label)
predict_number=ml.predict(test_vec)
verify_number=test_label
balance_accuracy, precision, recall, f1_value = plot_value(predict_number,verify_number)
save_performance(one_project+'-逻辑回归',balance_accuracy, precision, recall, f1_value)

c:\users\chang\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# KNN